First run everything in the submodule `predicting-poverty-replication`. Then:
- copy the `malawi_2016` folder into `data/LSMS` relative to the current directory
- copy 
- copy `predicting-poverty-replication/aggregated_feats.npy` to the current directory

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_stata('data/LSMS/malawi_2016/IHS4 Consumption Aggregate.dta')

In [3]:
PPP_2013 = 116.28

df = pd.read_stata('data/LSMS/malawi_2016/IHS4 Consumption Aggregate.dta')
df['persons_in_household'] = (df['rexpagg']/df['rexpaggpc']).astype(int)
df['annual_consumption_hh'] = df['rexpagg']
df['annual_consumption_hh'] /= PPP_2013 # accounting for purchasing power parity
df['annual_phone_consumption_hh'] = df['rexp_cat083']
df['annual_phone_consumption_hh'] = df['annual_phone_consumption_hh']/PPP_2013
df = df[['case_id', 'annual_consumption_hh', 'annual_phone_consumption_hh', 'persons_in_household']] # grab these columns

df_geo = pd.read_stata('data/LSMS/malawi_2016/HouseholdGeovariables_stata11/HouseholdGeovariablesIHS4.dta')
df_cords = df_geo[['case_id', 'HHID', 'lat_modified', 'lon_modified']]
df_cords.rename(columns={'lat_modified': 'lat', 'lon_modified': 'lon'}, inplace=True)

df_hhf = pd.read_stata('data/LSMS/malawi_2016/HH_MOD_F.dta')
df_hhf = df_hhf[['case_id', 'HHID', 'hh_f34', 'hh_f35']]
df_hhf.rename(columns={'hh_f34': 'cellphones_ph', 'hh_f35': 'estimated_annual_phone_cost_ph'}, inplace=True)


/Applications/anaconda3/envs/taddle/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [4]:
df = pd.merge(df, df_cords[['case_id', 'HHID']], on='case_id')

In [5]:
df_combined = pd.merge(df, df_cords, on=['case_id', 'HHID'])
df_combined = pd.merge(df_combined, df_hhf, on=['case_id', 'HHID'])

In [6]:
df_combined.shape

(12447, 9)

In [7]:
df_combined.head()

,case_id,annual_consumption_hh,annual_phone_consumption_hh,persons_in_household,HHID,lat,lon,cellphones_ph,estimated_annual_phone_cost_ph
0,301025230225,2560.039551,0.000000,3,0001c970eecf473099368557e2080b3e,-14.683761,34.915074,0.0,NaN
1,210374850204,14897.992188,139.863968,5,000509f5cfcc4b078a09672b09425e95,-14.005029,33.794591,1.0,1500.0
2,311057710075,4022.653809,222.308411,5,000bc107780044e59327dbf7ec960ac1,-16.826165,35.269503,2.0,2000.0
3,312048040073,3325.028320,0.000000,5,000d1d26325d4f73a2ffbb8a99ab4752,-15.004730,35.163219,0.0,NaN
4,311097790117,7148.121094,102.890274,7,00104e33315844fdb2b8c6fdd35912a1,-17.016698,35.079629,1.0,1000.0


In [8]:
df_combined['persons_in_household'].isna().sum()

0

In [9]:
df_stats = df_combined.copy()

In [10]:
data_cols = ['annual_consumption_hh', 'annual_phone_consumption_hh', 'cellphones_ph', 'estimated_annual_phone_cost_ph']

for c in data_cols:
    df_stats[c + '_na'] = df_stats[c].isna()

In [11]:
df_stats

,case_id,annual_consumption_hh,annual_phone_consumption_hh,persons_in_household,HHID,lat,lon,cellphones_ph,estimated_annual_phone_cost_ph,annual_consumption_hh_na,annual_phone_consumption_hh_na,cellphones_ph_na,estimated_annual_phone_cost_ph_na
0,301025230225,2560.039551,0.000000,3,0001c970eecf473099368557e2080b3e,-14.683761,34.915074,0.0,NaN,False,False,False,True
1,210374850204,14897.992188,139.863968,5,000509f5cfcc4b078a09672b09425e95,-14.005029,33.794591,1.0,1500.0,False,False,False,False
2,311057710075,4022.653809,222.308411,5,000bc107780044e59327dbf7ec960ac1,-16.826165,35.269503,2.0,2000.0,False,False,False,False
3,312048040073,3325.028320,0.000000,5,000d1d26325d4f73a2ffbb8a99ab4752,-15.004730,35.163219,0.0,NaN,False,False,False,True
4,311097790117,7148.121094,102.890274,7,00104e33315844fdb2b8c6fdd35912a1,-17.016698,35.079629,1.0,1000.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12442,205053630064,4161.387207,0.000000,5,ffe8b3cb44e646c4ba7e444c810b5262,-14.038697,34.511056,0.0,NaN,False,False,False,True
12443,307106820173,4508.041016,48.963737,2,ffe9da89ba74465ba9be8a7e8cbd77ce,-15.959665,35.034930,1.0,500.0,False,False,False,False
12444,208074540130,3620.645264,0.000000,5,ffe9e824c41f45c4aa5efb80d3ee9603,-14.263875,34.518640,0.0,NaN,False,False,False,True
12445,312048040036,2899.364258,0.000000,3,fffaa977174c43d79793cf945b57f948,-15.004730,35.163219,0.0,NaN,False,False,False,True


In [12]:
to_grab = ['lat', 'lon'] + [c + '_na' for c in data_cols]


clust_nas = df_stats.groupby(['lat', 'lon']).mean().reset_index()[to_grab]


In [13]:
clust_counts = df_stats.groupby(['lat', 'lon']).count().reset_index()[['lat', 'lon', 'persons_in_household']].rename(columns={'persons_in_household': 'num_hh_surveyed'})


In [14]:
df_clusters = df_combined.groupby(['lat', 'lon']).sum().reset_index()

for c in data_cols:
    # persons in household is now really all persons surveyed in cluster
    df_clusters[c[:-3] + '_pc'] = df_clusters[c] / df_clusters['persons_in_household']
    
df_clusters.drop(data_cols, axis=1, inplace=True)
df_clusters.rename(columns={'persons_in_household': 'persons_surveyed'}, inplace=True)

In [15]:
df_clusters.head()

,lat,lon,persons_surveyed,annual_consumption_pc,annual_phone_consumption_pc,cellphones_pc,estimated_annual_phone_cost_pc
0,-17.095150,35.217213,79,961.328026,47.627469,0.177215,428.481013
1,-17.092351,35.114643,70,855.258482,3.189638,0.028571,32.571429
2,-17.016698,35.079629,78,1058.343450,1.978659,0.025641,19.230769
3,-16.977243,35.205706,66,1127.493134,8.631155,0.045455,83.333333
4,-16.956385,35.168967,61,736.167585,5.081308,0.065574,49.180328


In [16]:
df_clusters.shape

(780, 7)

In [17]:
df_clusters = pd.merge(df_clusters, clust_nas, on=['lat', 'lon'])
df_clusters = pd.merge(df_clusters, clust_counts, on=['lat', 'lon'])

In [18]:
df_clusters.head()

,lat,lon,persons_surveyed,annual_consumption_pc,annual_phone_consumption_pc,cellphones_pc,estimated_annual_phone_cost_pc,annual_consumption_hh_na,annual_phone_consumption_hh_na,cellphones_ph_na,estimated_annual_phone_cost_ph_na,num_hh_surveyed
0,-17.095150,35.217213,79,961.328026,47.627469,0.177215,428.481013,0.0,0.0,0.0,0.5000,16
1,-17.092351,35.114643,70,855.258482,3.189638,0.028571,32.571429,0.0,0.0,0.0,0.8750,16
2,-17.016698,35.079629,78,1058.343450,1.978659,0.025641,19.230769,0.0,0.0,0.0,0.8750,16
3,-16.977243,35.205706,66,1127.493134,8.631155,0.045455,83.333333,0.0,0.0,0.0,0.8125,16
4,-16.956385,35.168967,61,736.167585,5.081308,0.065574,49.180328,0.0,0.0,0.0,0.7500,16


In [19]:
df_clusters.shape

(780, 12)

In [20]:
rename = {c: 'cluster_' + c for c in df_clusters.columns}
df_clusters.rename(columns=rename, inplace=True)

In [21]:
df_clusters.head()

,cluster_lat,cluster_lon,cluster_persons_surveyed,cluster_annual_consumption_pc,cluster_annual_phone_consumption_pc,cluster_cellphones_pc,cluster_estimated_annual_phone_cost_pc,cluster_annual_consumption_hh_na,cluster_annual_phone_consumption_hh_na,cluster_cellphones_ph_na,cluster_estimated_annual_phone_cost_ph_na,cluster_num_hh_surveyed
0,-17.095150,35.217213,79,961.328026,47.627469,0.177215,428.481013,0.0,0.0,0.0,0.5000,16
1,-17.092351,35.114643,70,855.258482,3.189638,0.028571,32.571429,0.0,0.0,0.0,0.8750,16
2,-17.016698,35.079629,78,1058.343450,1.978659,0.025641,19.230769,0.0,0.0,0.0,0.8750,16
3,-16.977243,35.205706,66,1127.493134,8.631155,0.045455,83.333333,0.0,0.0,0.0,0.8125,16
4,-16.956385,35.168967,61,736.167585,5.081308,0.065574,49.180328,0.0,0.0,0.0,0.7500,16


In [22]:
import geoio
filename = 'data/Nightlights/2013/F182013.v4c_web.stable_lights.avg_vis.tif'
img = geoio.GeoImage(filename)

In [23]:
im_array = np.squeeze(img.get_data())

In [24]:
import math

def create_space(lat, lon):
    # these are pulled from the paper to make the 10km^2 area
    return lat - (180/math.pi)*(5000/6378137), lon - (180/math.pi)*(5000/6378137)/math.cos(lat), \
            lat + (180/math.pi)*(5000/6378137), lon + (180/math.pi)*(5000/6378137)/math.cos(lat)

In [26]:
household_nightlights = []
for i,r in df_clusters.iterrows():
    min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)
    xminPixel, yminPixel = img.proj_to_raster(min_lon, min_lat)
    xmaxPixel, ymaxPixel = img.proj_to_raster(max_lon, max_lat)
    
    xminPixel, xmaxPixel = min(xminPixel, xmaxPixel), max(xminPixel, xmaxPixel)
    yminPixel, ymaxPixel = min(yminPixel, ymaxPixel), max(yminPixel, ymaxPixel)
    
    xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
    household_nightlights.append(im_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())

In [29]:
df_clusters['cluster_nightlights'] = household_nightlights

In [31]:
df_clusters.head()

,cluster_lat,cluster_lon,cluster_persons_surveyed,cluster_annual_consumption_pc,cluster_annual_phone_consumption_pc,cluster_cellphones_pc,cluster_estimated_annual_phone_cost_pc,cluster_annual_consumption_hh_na,cluster_annual_phone_consumption_hh_na,cluster_cellphones_ph_na,cluster_estimated_annual_phone_cost_ph_na,cluster_num_hh_surveyed,cluster_nightlights
0,-17.095150,35.217213,79,961.328026,47.627469,0.177215,428.481013,0.0,0.0,0.0,0.5000,16,0.000000
1,-17.092351,35.114643,70,855.258482,3.189638,0.028571,32.571429,0.0,0.0,0.0,0.8750,16,0.000000
2,-17.016698,35.079629,78,1058.343450,1.978659,0.025641,19.230769,0.0,0.0,0.0,0.8750,16,0.000000
3,-16.977243,35.205706,66,1127.493134,8.631155,0.045455,83.333333,0.0,0.0,0.0,0.8125,16,0.121212
4,-16.956385,35.168967,61,736.167585,5.081308,0.065574,49.180328,0.0,0.0,0.0,0.7500,16,0.502674


In [36]:
to_look = ['cluster_' + c[:-3] + '_pc' for c in data_cols] + ['cluster_nightlights']

df_clusters[to_look].corr()

,cluster_annual_consumption_pc,cluster_annual_phone_consumption_pc,cluster_cellphones_pc,cluster_estimated_annual_phone_cost_pc,cluster_nightlights
cluster_annual_consumption_pc,1.000000,0.547275,0.496989,0.532196,0.399983
cluster_annual_phone_consumption_pc,0.547275,1.000000,0.877638,0.970367,0.557845
cluster_cellphones_pc,0.496989,0.877638,1.000000,0.847314,0.571041
cluster_estimated_annual_phone_cost_pc,0.532196,0.970367,0.847314,1.000000,0.538049
cluster_nightlights,0.399983,0.557845,0.571041,0.538049,1.000000


In [37]:
df_clusters.to_csv('cluster_data.csv', index=False)